In [1]:
import pyvo.dal
from pyvo.auth import authsession, securitymethods
import getpass
import requests
import pyvo

In [2]:
# To avoid warnings
import warnings
warnings.simplefilter("ignore")

In [3]:
# Url of the TAP service
tap_url = "https://archive.cefca.es/catalogues/vo/tap/jpas-idr202406"

In [4]:
# Login
user = input("Username:")
pwd = getpass.getpass("Password:")
archive_login_url = "https://archive.cefca.es/catalogues/login"
login_args = {"login": user, "password": pwd,
              "submit": "Sign+In"}
login_header = {"Content-type": "application/x-www-form-urlencoded",
                "Accept": "text/plain"}


Username:lgutierrez
Password:········


In [5]:
pyvo.dal.tap.s = requests.Session()
response = pyvo.dal.tap.s.post(archive_login_url,
                 data=login_args, headers=login_header)

In [6]:
response.raise_for_status()

In [7]:
auth = authsession.AuthSession()
auth.credentials.set(securitymethods.ANONYMOUS, pyvo.dal.tap.s)


In [8]:
# Ejecutando la consulta
service = pyvo.dal.TAPService(tap_url, session=auth)
try:
    resultset = service.run_sync("SELECT COUNT(*) as total FROM jpas.MagABDualObj WHERE MAG_ERR_APER_6_0[jpas::J0660] < 0.4 AND MAG_ERR_APER_6_0[jpas::iSDSS] < 0.4")
except pyvo.DALQueryError as e:
    print(f"Error en consulta: {e}")

In [9]:
# Convertir a tabla Astropy
table = resultset.to_table()

In [10]:
# Obtener el número total de objetos
total_objects = table[0]["total"]
print("El total de objetos en el survey es:", total_objects)

El total de objetos en el survey es: 1792103


La tabla total tiene  12031905 objetos

### Probemos async

In [11]:
job = service.run_async("SELECT TOP 100 * FROM jpas.MagABDualObj WHERE MAG_ERR_APER_6_0[jpas::J0660] < 0.2 AND MAG_ERR_APER_6_0[jpas::iSDSS] < 0.2")


In [12]:
# Convertir a tabla de Astropy
table = job.to_table()
type(table)

astropy.table.table.Table

In [13]:
# Mostrar los primeros registros
print(table)

tile_id number    alpha_j2000     ... mag_aper_cor_6_0 mag_err_aper_cor_6_0
                      deg         ...       mag                            
------- ------ ------------------ ... ---------------- --------------------
   8955  10240 214.63259794839854 ... 22.708 .. 18.236       1.156 .. 0.005
   8955  10255 215.51029745851386 ...   99.0 .. 19.771        99.0 .. 0.016
   8955  10275 214.22434194821676 ...    99.0 .. 19.79        99.0 .. 0.016
   8955  10304   214.494161054305 ... 22.826 .. 20.025       1.159 .. 0.025
   8955  10305 214.58445157755116 ... 23.482 .. 20.748        2.36 .. 0.047
   8955  10317   215.515902883466 ... 22.098 .. 17.695       0.687 .. 0.003
   8955  10321 214.21229006429112 ... 22.139 .. 20.301       0.693 .. 0.026
   8955  10325 214.41684483508638 ...  22.99 .. 20.599       1.531 .. 0.033
   8955  10326  214.4390753561181 ... 21.917 .. 19.083       0.566 .. 0.009
    ...    ...                ... ...              ...                  ...
   8955  113

In [14]:
for i in table.columns:
    print(i)

tile_id
number
alpha_j2000
delta_j2000
hpix11
x_image
y_image
r_eff
fwhm_world
a_world
b_world
theta_j2000
isoarea_world
relerra_world
relerrb_world
errtheta_j2000
class_star
mag_auto
mag_iso
mag_petro
mu_max
petro_radius
kron_radius
mag_aper_0_8
mag_aper_1_0
mag_aper_1_2
mag_aper_1_5
mag_aper_2_0
mag_aper_3_0
mag_aper_4_0
mag_aper_6_0
mag_err_auto
mag_err_iso
mag_err_petro
mag_err_aper_0_8
mag_err_aper_1_0
mag_err_aper_1_2
mag_err_aper_1_5
mag_err_aper_2_0
mag_err_aper_3_0
mag_err_aper_4_0
mag_err_aper_6_0
flags
norm_wmap_val
mask_flags
single_detect
mag_aper_cor_3_0
mag_err_aper_cor_3_0
mag_aper_cor_6_0
mag_err_aper_cor_6_0


# Verlos filtros

In [15]:
resultset1 = service.run_sync("SELECT * FROM  jpas.Filter")  
print(resultset1.to_table())


filter_id  name wavelength width     kx     color_representation
                    AA       nm                                 
--------- ----- ---------- ----- ---------- --------------------
        1 uJAVA       3497   509 4.91599989              #CC00FF
        2 J0378       3782   157 4.56699991              #9900FF
        3 J0390       3904   150 4.44199991              #6F008C
        4 J0400       3996   148 4.34100008              #6900B0
        5 J0410       4110   144 4.24100018              #4300DE
        6 J0420       4203   147 4.14300013              #0000FF
        7 J0430       4303   148 4.04799986              #0069FF
        8 J0440       4403   152 3.95300007              #00B8FF
        9 J0450       4503   150 3.86100006              #00FFFF
      ...   ...        ...   ...        ...                  ...
       48 J0840       8429   148 1.59500003              #97003B
       49 J0850       8523   146 1.56099999              #910034
       50 J0860       862

In [16]:
filters = resultset1.to_table()

In [17]:
for i in filters:
    print(i["name"])

uJAVA
J0378
J0390
J0400
J0410
J0420
J0430
J0440
J0450
J0460
J0470
J0480
J0490
J0500
J0510
J0520
J0530
J0540
J0550
J0560
J0570
J0580
J0590
J0600
J0610
J0620
J0630
J0640
J0650
J0660
J0670
J0680
J0690
J0700
J0710
J0720
J0730
J0740
J0750
J0760
J0770
J0780
J0790
J0800
J0810
J0820
J0830
J0840
J0850
J0860
J0870
J0880
J0890
J0900
J0910
J1007
iSDSS


## Download the entire catalog

In [61]:
query = """
SELECT TOP 1000
    NUMBER,
    alpha_j2000,
    delta_j2000,
    tile_id,
    mag_aper_6_0[jpas::iSDSS] AS mag_iSDSS,      
    mag_aper_cor_6_0[jpas::iSDSS] AS mag_iSDSS_cor,  
    mag_err_aper_6_0[jpas::iSDSS] AS err_iSDSS,      
    mag_err_aper_cor_6_0[jpas::iSDSS] AS err_iSDSS_cor,
    mag_aper_6_0,          
    mag_aper_cor_6_0,      
    mag_err_aper_6_0,      
    mag_err_aper_cor_6_0,  
    class_star,
    flags,
    mask_flags
FROM 
    jpas.MagABDualObj 
WHERE 
    mag_err_aper_6_0[jpas::J0660] < 0.4 
    AND mag_err_aper_6_0[jpas::iSDSS] < 0.4
    AND mask_flags[jpas::J0660] = 0 
    AND mask_flags[jpas::iSDSS] = 0 
    AND flags[jpas::J0660] <= 3
    AND flags[jpas::iSDSS] <= 3
"""

In [62]:
main_data = service.run_async(query).to_table()


In [63]:
main_data

number,alpha_j2000,delta_j2000,tile_id,mag_isdss,mag_isdss_cor,err_isdss,err_isdss_cor,mag_aper_6_0,mag_aper_cor_6_0,mag_err_aper_6_0,mag_err_aper_cor_6_0,class_star,flags,mask_flags
,deg,deg,,,,,,mag,mag,,,,,
int32,float64,float64,int32,float32,float32,float32,float32,float32[57],float32[57],float32[57],float32[57],float32,int32[57],int32[57]
25130,338.6335978134234,22.418378321157284,8838,19.23475,19.180511,0.009650288,0.009650288,99.0 .. 19.235,99.0 .. 19.181,99.0 .. 0.01,99.0 .. 0.01,1.0,0 .. 0,0 .. 0
25131,338.6879789986762,22.416937200620136,8838,15.732048,15.677071,0.0010214603,0.0010214603,17.587 .. 15.732,17.453 .. 15.677,0.011 .. 0.001,0.011 .. 0.001,0.999,0 .. 0,0 .. 0
25136,339.4323904016746,22.418424930737444,8838,19.506378,19.450806,0.011075052,0.011075052,22.735 .. 19.506,22.572 .. 19.451,0.969 .. 0.011,0.969 .. 0.011,0.03,0 .. 0,0 .. 0
25138,339.2173259076366,22.41882242514427,8838,20.312582,20.260983,0.021470856,0.021470856,99.0 .. 20.313,99.0 .. 20.261,99.0 .. 0.021,99.0 .. 0.021,0.962,0 .. 0,0 .. 0
25140,339.34243761597725,22.418529517107938,8838,19.751074,19.696915,0.014181368,0.014181368,23.803 .. 19.751,23.652 .. 19.697,2.63 .. 0.014,2.63 .. 0.014,0.812,0 .. 0,0 .. 0
25143,339.47908613005967,22.417901916935936,8838,18.27458,18.219196,0.004218377,0.004218377,99.0 .. 18.275,99.0 .. 18.219,99.0 .. 0.004,99.0 .. 0.004,0.901,0 .. 0,0 .. 0
25144,339.5001273487729,22.418304206356023,8838,19.795458,19.743582,0.013659507,0.013659507,22.666 .. 19.795,22.503 .. 19.744,0.914 .. 0.014,0.914 .. 0.014,0.904,0 .. 0,0 .. 0
25147,339.096093615288,22.418972597136975,8838,21.007702,20.957994,0.040489793,0.040489793,23.1 .. 21.008,22.95 .. 20.958,1.105 .. 0.04,1.105 .. 0.04,0.976,0 .. 0,0 .. 0
